In [3]:
import numpy as np
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import os

In [4]:
df = pd.read_csv("C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Datos/Datos2011Limipios.csv")

In [5]:
def create_encoders_and_transform(df, categorical_columns):
    """
    Crea encoders para binarias y dummies para categóricas de más de 2 clases.
    Devuelve el DataFrame procesado, diccionario de encoders y columnas finales.
    """
    encoders = {}
    df_encoded = df.copy()

    binary_columns = []
    multi_columns = []

    # Detectar cuáles columnas son binarias o multinivel
    for col in categorical_columns:
        if col in df_encoded.columns:
            # Limpiar valores nulos y convertir a string
            df_encoded[col] = df_encoded[col].fillna('unknown').astype(str)
            unique_values = df_encoded[col].nunique()
            
            if unique_values == 2:
                binary_columns.append(col)
            elif unique_values > 2:
                multi_columns.append(col)

    # Codificar las binarias con LabelEncoder
    for col in binary_columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        encoders[col] = le

    # Crear dummies para las multinivel
    if multi_columns:
        # Obtener las columnas a convertir en dummies
        df_multi = df_encoded[multi_columns].copy()
        
        # Crear dummies
        df_dummies = pd.get_dummies(df_multi, prefix=multi_columns, drop_first=True, dtype=int)
        
        # Eliminar columnas originales y agregar dummies
        df_encoded = df_encoded.drop(columns=multi_columns)
        df_encoded = pd.concat([df_encoded, df_dummies], axis=1)
    
    # Asegurar que todas las columnas sean numéricas
    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object':
            try:
                # Intentar convertir a numérico
                df_encoded[col] = pd.to_numeric(df_encoded[col], errors='coerce')
                # Llenar NaN con 0
                df_encoded[col] = df_encoded[col].fillna(0)
            except:
                # Si falla, usar LabelEncoder como último recurso
                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
                encoders[f"{col}_emergency"] = le
    
    # Convertir todas las columnas a float32 para consistencia
    for col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].astype('float32')

    return df_encoded, encoders

In [6]:
def transform_new_data(new_data, encoders, reference_columns):
    """
    Transforma nuevos datos con los mismos encoders y columnas que el entrenamiento.
    """
    new_data_encoded = new_data.copy()
    
    # Identificar columnas binarias (que tienen encoder) y multinivel
    binary_columns = [col for col in encoders.keys() if not col.endswith('_emergency')]
    
    # Limpiar y preparar datos
    for col in new_data_encoded.columns:
        if new_data_encoded[col].dtype == 'object':
            new_data_encoded[col] = new_data_encoded[col].fillna('unknown').astype(str)

    # Aplicar LabelEncoder a columnas binarias
    for col in binary_columns:
        if col in new_data_encoded.columns:
            try:
                # Manejar valores no vistos durante el entrenamiento
                unique_values = set(new_data_encoded[col].unique())
                encoder_classes = set(encoders[col].classes_)
                
                # Si hay valores nuevos, mapearlos al primer valor conocido
                if not unique_values.issubset(encoder_classes):
                    unknown_values = unique_values - encoder_classes
                    for unknown_val in unknown_values:
                        new_data_encoded[col] = new_data_encoded[col].replace(
                            unknown_val, encoders[col].classes_[0]
                        )
                
                new_data_encoded[col] = encoders[col].transform(new_data_encoded[col])
            except Exception as e:
                print(f"Error en columna {col}: {e}")
                new_data_encoded[col] = 0  # Valor por defecto

    # Identificar columnas multinivel de los datos de referencia
    dummy_columns = [col for col in reference_columns 
                    if any(col.startswith(f"{bc}_") for bc in binary_columns) == False]
    
    # Crear dummies para columnas multinivel
    multi_columns_in_data = []
    for ref_col in dummy_columns:
        # Extraer el nombre base de la columna dummy
        for original_col in new_data_encoded.columns:
            if ref_col.startswith(f"{original_col}_") and original_col not in binary_columns:
                if original_col not in multi_columns_in_data:
                    multi_columns_in_data.append(original_col)
    
    if multi_columns_in_data:
        # Crear dummies
        df_multi = new_data_encoded[multi_columns_in_data].copy()
        
        # Asegurar que los valores sean strings
        for col in multi_columns_in_data:
            df_multi[col] = df_multi[col].astype(str)
        
        df_dummies = pd.get_dummies(df_multi, prefix=multi_columns_in_data, drop_first=True, dtype=int)
        
        # Eliminar columnas originales y agregar dummies
        new_data_encoded = new_data_encoded.drop(columns=multi_columns_in_data, errors='ignore')
        new_data_encoded = pd.concat([new_data_encoded, df_dummies], axis=1)

    # Asegurar que tenga todas las columnas de referencia
    for col in reference_columns:
        if col not in new_data_encoded.columns:
            new_data_encoded[col] = 0

    # Reindexar con las columnas de referencia
    new_data_encoded = new_data_encoded.reindex(columns=reference_columns, fill_value=0)
    
    # Asegurar que todas las columnas sean numéricas
    for col in new_data_encoded.columns:
        if new_data_encoded[col].dtype == 'object':
            try:
                new_data_encoded[col] = pd.to_numeric(new_data_encoded[col], errors='coerce')
                new_data_encoded[col] = new_data_encoded[col].fillna(0)
            except:
                # Como último recurso, convertir usando un encoder temporal
                le_temp = LabelEncoder()
                new_data_encoded[col] = le_temp.fit_transform(new_data_encoded[col].astype(str))
    
    # Convertir todas las columnas a float32 para consistencia
    for col in new_data_encoded.columns:
        new_data_encoded[col] = new_data_encoded[col].astype('float32')

    return new_data_encoded

In [11]:
categorical_columns = ['cole_depto_ubicacion', 'cole_jornada', 
                      'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 
                      'fami_estratovivienda']

In [12]:
df_processed, encoders = create_encoders_and_transform(df, categorical_columns)

In [13]:
feature_columns = [col for col in df_processed.columns if col != 'resultado']
X = df_processed[feature_columns]
y = df['resultado']
y = y.map({'no_exitoso': 0, 'exitoso': 1})

In [14]:
columns_info = {
    'feature_columns': list(X.columns),
    'encoders': encoders,
    'categorical_columns': categorical_columns
}

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = {cls: weight for cls, weight in zip(np.unique(y), weights)}

In [17]:
def create_model(input_dim, hidden_units, learning_rate):
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_dim=input_dim))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

In [18]:
mlflow.set_tracking_uri('http://localhost:5000')
experiment_name = "icfes2011"

In [19]:
try:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Experimento creado con ID: {experiment_id}")
except mlflow.exceptions.MlflowException:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    print(f"Usando experimento existente con ID: {experiment_id}")

Usando experimento existente con ID: 690124376614287936


In [20]:
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/690124376614287936', creation_time=1748128300681, experiment_id='690124376614287936', last_update_time=1748128300681, lifecycle_stage='active', name='icfes2011', tags={}>

In [21]:
batch_sizes = [30, 120, 200]
epochs_list = [5, 10, 15]
learning_rates = [0.001, 0.01, 0.05]
hidden_units_list = [300, 400, 500]

In [22]:
best_accuracy = 0
best_model_info = {}
best_run_id = ""

In [23]:
os.makedirs("modelos_guardados", exist_ok=True)

In [ ]:
exp = 0
total_experiments = len(batch_sizes) * len(epochs_list) * len(learning_rates) * len(hidden_units_list)

for batch_size in batch_sizes:
    for epochs in epochs_list:
        for lr in learning_rates:
            for hidden_units in hidden_units_list:
                exp += 1
                print(f"Experimento {exp}/{total_experiments}: batch_size={batch_size}, epochs={epochs}, learning_rate={lr}, hidden_units={hidden_units}")
                
                with mlflow.start_run(run_name=f"exp_{exp}") as run:
                    # Crear y entrenar modelo
                    model = create_model(X_train.shape[1], hidden_units, lr)
                    
                    history = model.fit(X_train, y_train, 
                                      epochs=epochs, 
                                      batch_size=batch_size,
                                      class_weight=class_weights, 
                                      verbose=0, 
                                      validation_data=(X_test, y_test))
                    
                    # Evaluar modelo
                    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
                    
                    # Log parámetros y métricas
                    mlflow.log_param("batch_size", batch_size)
                    mlflow.log_param("epochs", epochs)
                    mlflow.log_param("learning_rate", lr)
                    mlflow.log_param("hidden_units", hidden_units)
                    mlflow.log_param("input_dim", X_train.shape[1])
                    
                    mlflow.log_metric("test_loss", loss)
                    mlflow.log_metric("test_accuracy", accuracy)
                    mlflow.log_metric("final_train_loss", history.history['loss'][-1])
                    mlflow.log_metric("final_val_loss", history.history['val_loss'][-1])
                    mlflow.log_metric("final_train_accuracy", history.history['accuracy'][-1])
                    mlflow.log_metric("final_val_accuracy", history.history['val_accuracy'][-1])
                    
                    # Guardar modelo en MLflow
                    mlflow.keras.log_model(model, "model")
                    
                    # Guardar información adicional como artefactos
                    with open("temp_columns_info.pkl", "wb") as f:
                        pickle.dump(columns_info, f)
                    mlflow.log_artifact("temp_columns_info.pkl", "preprocessing_info")
                    
                    # Rastrear el mejor modelo
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_run_id = run.info.run_id
                        best_model_info = {
                            'run_id': run.info.run_id,
                            'accuracy': accuracy,
                            'loss': loss,
                            'batch_size': batch_size,
                            'epochs': epochs,
                            'learning_rate': lr,
                            'hidden_units': hidden_units
                        }
                        
                        # Guardar mejor modelo localmente también
                        model.save("modelos_guardados/mejor_modelo.keras")
                        with open("modelos_guardados/columns_info.pkl", "wb") as f:
                            pickle.dump(columns_info, f)
                        
                        print(f"   Nuevo mejor modelo. Accuracy: {accuracy:.4f}")

Experimento 1/81: batch_size=30, epochs=5, learning_rate=0.001, hidden_units=300


c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/05/24 18:29:13 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/24 18:29:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   Nuevo mejor modelo. Accuracy: 0.9649
🏃 View run exp_1 at: http://localhost:5000/#/experiments/690124376614287936/runs/824925c7b06c4354bd2fac95f67fab07
🧪 View experiment at: http://localhost:5000/#/experiments/690124376614287936
Experimento 2/81: batch_size=30, epochs=5, learning_rate=0.001, hidden_units=400


c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/05/24 18:30:46 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/24 18:30:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   Nuevo mejor modelo. Accuracy: 0.9800
🏃 View run exp_2 at: http://localhost:5000/#/experiments/690124376614287936/runs/e79bbbd03b2e4660ba0baaf4e1b9719d
🧪 View experiment at: http://localhost:5000/#/experiments/690124376614287936
Experimento 3/81: batch_size=30, epochs=5, learning_rate=0.001, hidden_units=500


...

c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/05/24 21:21:57 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/24 21:22:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run exp_79 at: http://localhost:5000/#/experiments/690124376614287936/runs/1de822e19de44aef960f7a625fc47a14
🧪 View experiment at: http://localhost:5000/#/experiments/690124376614287936
Experimento 80/81: batch_size=200, epochs=15, learning_rate=0.05, hidden_units=400


c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/05/24 21:22:49 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/24 21:22:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run exp_80 at: http://localhost:5000/#/experiments/690124376614287936/runs/b2386d1c7a4f4da4bb576160e8ea02e4
🧪 View experiment at: http://localhost:5000/#/experiments/690124376614287936
Experimento 81/81: batch_size=200, epochs=15, learning_rate=0.05, hidden_units=500


c:\Users\sofia\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/05/24 21:23:43 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
